In [39]:
# hold to your courage hold to your wits
# Imports
import torch

import torch.nn as nn
import torch
import torch.optim as optim
from torchvision import transforms
from tqdm import tqdm
#from torchsummary import summary
import math

import torch.nn.functional as F

In [40]:
#device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [41]:
img_channel=3
img_height=32
img_width=100
num_class=128+1
map_to_seq_hidden=64
rnn_hidden=128 + 15

In [42]:
class ECABlock(nn.Module):
    def __init__(self, channels, k_size=3):
        super(ECABlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv1d = nn.Conv1d(1, 1, kernel_size=k_size, padding=(k_size - 1) // 2, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        y = self.avg_pool(x)  
        y = y.squeeze(-1).transpose(-1, -2)  # (bs, c, 1) -> (bs, 1, c)
        y = self.conv1d(y)  
        y = self.sigmoid(y)  
        y = y.transpose(-1, -2).unsqueeze(-1)  #  (bs, 1, c) - (bs, c, 1, 1)
        y = y.expand_as(x)  
        return x * y  # why this?


In [43]:
# Model

class CRNN_2(nn.Module):

    def __init__(self, img_channel=3, img_height=28, img_width=128, num_class=128+1,
                 map_to_seq_hidden=64, rnn_hidden=128, leaky_relu=False):
        super(CRNN_2, self).__init__()
        

        self.cnn, (output_channel, output_height, output_width) = \
            self._cnn_backbone(img_channel, img_height, img_width, leaky_relu)

        self.map_to_seq = nn.Linear(output_channel * output_height, map_to_seq_hidden)

        self.rnn1 = nn.LSTM(map_to_seq_hidden, rnn_hidden, bidirectional=True )
        self.rnn2 = nn.LSTM(2 * rnn_hidden, rnn_hidden, bidirectional=True,num_layers=2,dropout=0.5) #,num_layers=2,dropout=0.4

        self.dense = nn.Linear(2 * rnn_hidden, num_class)

    def _cnn_backbone(self, img_channel, img_height, img_width, leaky_relu):
        assert img_height % 16 == 0
        assert img_width % 4 == 0

        channels = [img_channel, 64, 128, 256, 256,350, 400 ,400]
        kernel_sizes = [3, 3, 3, 3, 3, 3,2]
        strides = [1, 1, 1,1, 1,1 ,1]
        paddings = [1, 1, 1, 1 ,1,1, 0]

        cnn = nn.Sequential()

        def conv_relu(i, batch_norm=False):
            # shape of input: (batch, input_channel, height, width)
            input_channel = channels[i]
            output_channel = channels[i+1]

            cnn.add_module(
                f'conv{i}',
                nn.Conv2d(input_channel, output_channel, kernel_sizes[i], strides[i], paddings[i])
            )

            if batch_norm:
                cnn.add_module(f'batchnorm{i}', nn.BatchNorm2d(output_channel))

            relu = nn.LeakyReLU(0.2, inplace=True) if leaky_relu else nn.ReLU(inplace=True)
            cnn.add_module(f'relu{i}', relu)
            if output_channel > 256:
                cnn.add_module(f'attention{i}',ECABlock(output_channel))

        # size of image: (channel, height, width) = (img_channel, img_height, img_width)
        conv_relu(0)
        
        cnn.add_module('pooling0', nn.MaxPool2d(kernel_size=2, stride=2))
        # (64, img_height // 2, img_width // 2)

        conv_relu(1)
        cnn.add_module('dropout0',nn.Dropout2d(p=0.2)) # Dropout
        cnn.add_module('pooling1', nn.MaxPool2d(kernel_size=2, stride=2))
        # (128, img_height // 4, img_width // 4)

        conv_relu(2)
        
        cnn.add_module('dropout1',nn.Dropout2d(p=0.2)) # Dropout
        conv_relu(3)
        cnn.add_module('dropout2',nn.Dropout2d(p=0.3))
        cnn.add_module(
            'pooling2',
            nn.MaxPool2d(kernel_size=(2, 1))
        )  # (256, img_height // 8, img_width // 4)

        conv_relu(4, batch_norm=True)
        cnn.add_module('dropout2',nn.Dropout2d(p=0.4))
        conv_relu(5, batch_norm=True)
        cnn.add_module('dropout2',nn.Dropout2d(p=0.4))
        cnn.add_module(
            'pooling3',
            nn.MaxPool2d(kernel_size=(2, 1))
        )  # (512, img_height // 16, img_width // 4)

        conv_relu(6)  # (512, img_height // 16 - 1, img_width // 4 - 1)

        output_channel, output_height, output_width = \
            channels[-1], img_height // 16-1 , img_width // 4 -1
        return cnn, (output_channel, output_height, output_width)

    def forward(self, images):
        # shape of images: (batch, channel, height, width)

        conv = self.cnn(images)
        batch, channel, height, width = conv.size()

        conv = conv.view(batch, channel * height, width)
        conv = conv.permute(2, 0, 1)  # (width, batch, feature)
        seq = self.map_to_seq(conv)

        recurrent, _ = self.rnn1(seq)
        recurrent, _ = self.rnn2(recurrent)

        output = self.dense(recurrent)
        return output  # shape: (seq_len, batch, num_class)

In [44]:
# load pretrained model if available
#model_2 = torch.load('model_2.pth')

In [45]:
model_2 = CRNN_2( img_channel, img_height, img_width, num_class,map_to_seq_hidden, rnn_hidden).to(device)

In [46]:
# transform
transform= transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((32,128)),
    transforms.RandomRotation(degrees=20),
])

In [47]:
# Hyperparams
batch_size =512
lr = 0.0003
epochs = 200
criterion = nn.CTCLoss()
optimiser = optim.Adam(model_2.parameters(),lr=lr, weight_decay=1e-5)
#optimiser = torch.optim.AdamW(model_2.parameters(), lr=0.0003)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimiser, 'min', patience=10, factor=0.1)


In [48]:
#data loders
import os
from PIL import Image
from torch.utils.data import Dataset,random_split
# from matplotlib import pyplot as plt

class CustomImageTextDataset(Dataset):
    def __init__(self, image_dir, image_names, texts, transform=None):
        self.image_dir = image_dir
        self.image_names = image_names
        self.texts = texts
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_path =    self.image_names[idx]
        image = Image.open(img_path).convert('RGB')
        text = self.texts[idx]

        if self.transform:
            image = self.transform(image)

        return image, text

##
class mydataset:
    def __init__(self,data_dir,transform=None,val=False):
        #paths to the directories/files
        extraction_path = data_dir
        image_dir = os.path.join(extraction_path, 'images')
        text_file_path = os.path.join(extraction_path, 'train_labels.txt' if val==False else 'val_labels.txt')
        image_names_file_path = os.path.join(extraction_path, 'train_images.txt' if val==False else 'val_images.txt')

        #read the image names
        with open(image_names_file_path, 'r',encoding='utf-8') as f:
            image_names = f.read().splitlines()

        #read the texts
        with open(text_file_path, 'r',encoding='utf-8') as f:
            texts = f.read().splitlines()

        # ensure that image_names and texts have the same length
        assert len(image_names) == len(texts), "Mismatch between image names and texts"
        # retrurn object of you clss
        self.dataset= CustomImageTextDataset(image_dir=image_dir, image_names=image_names, texts=texts, transform=transform)

    def get_dataset(self):
        return self.dataset

# # dataset
from torch.utils.data import DataLoader


#dataset->train
data_train = mydataset(data_dir='train',transform=transform)
data_train = data_train.get_dataset()
data_train,_ = random_split(data_train,[len(data_train)//1,len(data_train)-len(data_train)//1])

#dataset->val
data_val = mydataset(data_dir='val',transform=transform,val=True)
data_val = data_val.get_dataset()

 #dataloaders
trainloader = DataLoader(data_train,shuffle=True,batch_size=batch_size)
valloader = DataLoader(data_val,shuffle=True,batch_size=batch_size)

In [49]:
print(valloader.__len__())
trainloader.__len__()

5


35

In [50]:
# Label Transformer
class LabelTransformer():
    """
    encoder and decoder

    Args:
        letters (str): Letters contained in the data
    """

    def __init__(self, letters= "ಂಃಅಆಇಈಉಊಋಌಎಏಐಒಓಔಕಖಗಘಙಚಛಜಝಞಟಠಡಢಣತಥದಧನಪಫಬಭಮಯರಱಲಳವಶಷಸಹ಼ಽಾಿೀುೂೃೄೆೇೈೊೋೌ್ೕೖೞೠೡೢೣ೦೧೨೩೪೫೬೭೮೯ೱೲ"):
        self.encode_map = {letter: idx+1 for idx, letter in enumerate(letters)}
        self.decode_map = ' ' + letters

    # def encode(self, text):
    #     if isinstance(text, str):
    #         length = [len(text)]
    #         result = [self.encode_map[letter] for letter in text]
    #     else:
    #         length = []
    #         result = []
    #         for word in text:
    #             length.append(len(word))
    #             result.extend([self.encode_map[letter] for letter in word])
    #     return torch.IntTensor(result), torch.IntTensor(length)
    def encode(self, text):
        result = []
        length = []
        for word in text:
            length.append(len(word))
            for letter in word:
                if letter in self.encode_map:
                    result.append(self.encode_map[letter])
                else:
                    # Log the problematic letter and the word it was part of
                    # print(f"Warning: Character '{letter}' not found in encode_map. In word: '{word}'")
                    # Optionally, handle the unknown character
                    result.append(self.encode_map.get('unknown', 0))  # Use a default value or handle as needed
        return torch.IntTensor(result), torch.IntTensor(length)

    def decode(self, text_code, length):
        result = []
        idx = 0
        for len in length:
            word = []
            for i in range(len):
                if text_code[idx] != 0 and (i == 0 or text_code[idx] != text_code[idx - 1]):
                    word.append(self.decode_map[text_code[idx]])
                idx += 1
            result.append(''.join(word))
        return result


In [51]:
# CTC decoder
def ctc_decoder(predictions, label_transformer, blank=0):
    decoded_output = []
    batch_size = predictions.size(1)
    max_seq_length = predictions.size(0)

    # Get the indices of the max probabilities (predictions)
    _, max_indices = torch.max(predictions, 2)

    for batch in range(batch_size):
        pred_string = []
        previous_char = None
        for t in range(max_seq_length):
            current_char = max_indices[t][batch].item()
            if current_char != blank and current_char != previous_char:
                pred_string.append(current_char)
            previous_char = current_char
        decoded_output.append(''.join([label_transformer.decode_map[idx] for idx in pred_string if idx != blank]))

    return decoded_output

In [52]:
# val acc
def val_acc(model,loader,labeltransform):
  model.eval()
  with torch.no_grad():
    total_correct = 0
    total_samples = 0
    for data, target in tqdm(loader):
        data = data.to(device)

        pred = model(data)  # Model output shape: (seq_len, batch_size, num_classes)

        # Decode predictions
        pred_decoded = ctc_decoder(pred, labeltransform)

        # Compute the number of correct predictions
        correct_predictions = sum(p == t for p, t in zip(pred_decoded, target))
        total_correct += correct_predictions
        total_samples += len(target)

    # Calculate accuracy
    acc = total_correct *100/ total_samples
    print(f'Validation Accuracy: {acc:.4f}')
    return acc


In [53]:
# # Training Loop goes here
# loss=[]
# epoch_loss=0
# labeltransform = LabelTransformer()

# for epoch in range(epochs):
#     model_2.train()
#     print("============== epoch :",epoch,"=============")
#     for idx,(data,targets) in tqdm(enumerate(trainloader)):
#         data = data.to(device)
#         x = targets
#         # Output
#         targets,target_length = labeltransform.encode(targets)
#         optimiser.zero_grad()
#         # out
#         out = model_2(data)
#         out_length = torch.IntTensor([out.size(0)]*out.size(1))

#         # Loss
#         loss = criterion(F.log_softmax(out,2),targets,out_length,target_length)
#         loss.backward()
#         # Gradient Clipping
#         #torch.nn.utils.clip_grad_norm_(model_2.parameters(), max_norm=1)
#         # Optim
#         optimiser.step()

#         epoch_loss += loss.item()
#     # After epoch loss calculation
#     scheduler.step(epoch_loss / len(trainloader))
#     print("epoch :",epoch,"loss",epoch_loss/len(trainloader))

#     if(epoch % 5 == 0):
#       acc = val_acc(model_2,valloader,labeltransform)
#       val_acc(model_2,trainloader,labeltransform)
#     if(acc > 75):
#         #save the trained model
#         torch.save(model_2.state_dict(), 'model_75+.pth')
#         break
#     loss.append(epoch_loss)
#     epoch_loss=0



# Training Loop goes here
epoch_losses=[]
epoch_loss=0
labeltransform = LabelTransformer()

for epoch in range(epochs):
    model_2.train()
    print("============== epoch :",epoch,"=============")
    for idx,(data,targets) in tqdm(enumerate(trainloader)):
        data = data.to(device)
        x = targets
        # Output
        targets,target_length = labeltransform.encode(targets)
        optimiser.zero_grad()
        # out
        out = model_2(data)
        out_length = torch.IntTensor([out.size(0)]*out.size(1))

        # Loss
        loss_value = criterion(F.log_softmax(out,2),targets,out_length,target_length)
        loss_value.backward()
        # Gradient Clipping
        #torch.nn.utils.clip_grad_norm_(model_2.parameters(), max_norm=1)
        # Optim
        optimiser.step()

        epoch_loss += loss_value.item()
    # After epoch loss calculation
    scheduler.step(epoch_loss / len(trainloader))
    print("epoch :",epoch,"loss",epoch_loss/len(trainloader))

    if(epoch % 5 == 0):
      acc = val_acc(model_2,valloader,labeltransform)
      val_acc(model_2,trainloader,labeltransform)
    if(acc > 75):
        #save the trained model
        torch.save(model_2.state_dict(), 'model_75+.pth')
        break
    epoch_losses.append(epoch_loss)
    epoch_loss=0

============== epoch : 0 =============


35it [03:52,  6.66s/it]


epoch : 0 loss 12.018446241106306


100%|██████████| 5/5 [00:30<00:00,  6.07s/it]


Validation Accuracy: 0.0000


...

35it [06:15, 10.72s/it]


epoch : 81 loss 3.7159846782684327
============== epoch : 82 =============


35it [12:59, 22.27s/it] 


epoch : 82 loss 3.718151208332607
============== epoch : 83 =============


28it [37:29, 16.07s/it] 

In [16]:
#save the trained model
torch.save(model_2.state_dict(), 'model_76.1.pth')

In [17]:
print(val_acc(model_2,valloader,labeltransform))
val_acc(model_2,trainloader,labeltransform)

  0%|          | 0/5 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'val/images/15122022_L_GH042772_image_000103_23.png'